In [ ]:
import os.path
css = ""
if os.path.isfile("style.html"):
    from IPython.core.display import HTML
    with open("style.html", "r") as file:
        css = file.read()
HTML(css)

In [ ]:
%run ./nmm-game.ipynb
%run ./nmm-heuristic.ipynb

## Minimax

Der *Minimax* Algorithmus bietet eine einfache Möglichkeit, um den perfekten Weg in einem Nullsummenspiel zu berechnen. Um diesen zu finden, wird der komplette Spielbaum vollständig per Tiefensuche durchsucht.
Für einen Spielbaum mit der Tiefe `h` und dem Verzweigungsgrad `b` bedeutet das für die Zeit `t` und den Speicher `m`:
<font size="4">
$$ t_{Minimax} \in \mathcal{O}(b^h) $$
$$ m_{Minimax} \in \mathcal{O}(b\cdot h) $$
</font>
Logischerweise ist *Minimax* somit nicht für die komplette Berechnung des Spiels geeignet, weil dies die üblicherweise zur Verfügung stehenden Ressourcen überschreitet. Aus diesem Grund wird die Tiefensuche auf eine maximale Tiefe `limit` beschränkt. Dies hat jedoch zur Folge, dass bei der maximalen Tiefe häufig noch kein eindeutiges Ergebnis *Mini* dem Wert `-1` (sichere Niederlage für `player`) oder *Max* mit dem Wert `1` (sicherer Sieg für `player`) erkannt werden konnte. Somit ist die Funktion `heuristic(state, player)` notwendig die in einem solchen Fall eine einfache heuristische Bewertung des `state` für den `player` durchführt und einen Wert $ -1 < value < 1 $ berechnet.

Die Funktion `memoize(f)` nimmt eine Funktion `f` und überprüft, ob diese bereits mit den beiden gleichen Haupt-Parametern `state` und `player` aufgerufen wurde und liefert den gespeicherten Antwortwert. Wenn die Funktion bisher nicht aufgerufen wurde, werden die Parameter und das entsprechende Ergebnis in den Cache gespeichert.

In [ ]:
Cache = {}

def memoize(f):
    global Cache

    def f_memoized(*args):
        key = (args[0], args[1])
        if key in Cache:
            return Cache[key]
        result = f(*args)
        Cache[key] = result
        return result

    return f_memoized

Die Funktion `value_MM` nimmt einen Spielzustand `state` und liefert bei Ende des Spiels den Wert `value` mit Hilfe der `utility(state, player)`-Funktion für den Spieler `player`. 
Mit `limit` wird die Rekursionstiefe begrenzt. Beim Erreichen dieser wird die Funktion `heuristic(state, player)` aufgerufen.
In allen weiteren Fällen wird rekursiv in den nächsten möglichen Schritten nach dem maximal zu erreichenden Wert `value` gesucht. Zusammengefasst bedeutet das für die Funktion:

$$ value\_MM(state, player, limit) = \begin{cases}
utility(state, player) & falls & finished(state, player) = true \\
heuristic(state, player) & falls & limit = 0 \\
max(-value\_MM(ns, \neg{player}, limit))  \forall  ns \in nextStates(state, player) & sonst
\end{cases}
$$

In [ ]:
@memoize
def value_MM(state, player, limit):
    if finished(state, player):
        return utility(state, player)
    if limit == 0:
        return heuristic(state, player)
    return max([-value_MM(ns, opponent(player), limit-1) for ns in nextStates(state, player)])

Die Funktion `bestMove` berechnet den günstigsten Zug für einen Spieler `player` um das Spiel zu gewinnen. Dieser wird innerhalb der maximalen Rekursionstiefe `limit` mit dem gegebenen Ausgangszustand `state` gesucht

$$ bestMove\_MM(state, player, limit) = max(-value\_MM(ns, \neg{player}, limit))  \forall  ns \in nextStates(state, player) $$

In [ ]:
def bestMove_MM(state, player, limit = None):
    if limit is None:
        limit = 2
    # Clear cache
    global Cache
    Cache = {}

    moves = [
        (-value_MM(state, opponent(player), limit), state)
        for state in nextStates(state, player)
    ]
    return max(moves, key=lambda m: m[0])